# Indexing

We will demonstrate the raster index with a simple dataset with  rectilinear coordinates and no rotation or skew.
Both x and y coordinates are 1-dimensional.

In [ ]:
%xmode minimal

import numpy as np
import xarray as xr

import rasterix

np.set_printoptions(threshold=10, edgeitems=2)
xr.set_options(display_expand_indexes=True)

In [ ]:
source = "https://noaadata.apps.nsidc.org/NOAA/G02135/south/daily/geotiff/2024/01_Jan/S_20240101_concentration_v4.0.tif"

da = xr.open_dataarray(source, engine="rasterio").pipe(rasterix.assign_index)
da

## Positional Indexing (`isel`)

```{seealso}
Now we will demonstrate indexing with a `RasterIndex`. See the Xarray [docs](https://docs.xarray.dev/en/stable/user-guide/indexing.html) and [tutorial](https://tutorial.xarray.dev/fundamentals/02.1_indexing_Basic.html) to understand the concepts underlying this section.
```

### Slicing both x and y

Slicing preserves the laziness of coordinates since the new dataset can be represented by a new affine transform.

In [ ]:
da_sliced = da.isel(x=slice(1, 4), y=slice(None, None, 2))
da_sliced

Compare the transforms before/after slicing.

In [ ]:
(da.xindexes["x"].transform(), da_sliced.xindexes["x"].transform())

In [ ]:
print(da_sliced.xindexes["x"])
print(da_sliced.xindexes["y"])

### Outer indexing with array-like indexers

In [ ]:
da_outer = da.isel(x=[0, 2, 4], y=[0, 0, 1])
da_outer

We cannot compute a new affine transform given arbitrary array positions, so the resulting dataset has no indexes associated with `x` and `y`.

In [ ]:
da_outer.xindexes

### Basic indexing with scalars

In [ ]:
da_scalar = da.isel(x=0, y=1)
da_scalar

In [ ]:
da_xscalar = da.isel(x=0)
da_xscalar

In [ ]:
# da_xscalar.xindexes["y"]  # should return an index

### Vectorized (fancy) indexing

```{seealso}
See the Xarray [tutorial](https://tutorial.xarray.dev/intermediate/indexing/advanced-indexing.html) for more on this topic.
```

Indexing the spatial coordinates with Xarray `Variable`  or `DataArray` objects returns an unindexed object. The result cannot be represented by a RasterIndex in general.

In [ ]:
da_points = da.isel(x=xr.DataArray([0, 1], dims="z"), y=xr.DataArray([1, 1], dims="z"))
da_points

In [ ]:
da_points2d = da.isel(
    x=xr.Variable(("u", "v"), [[0, 1], [2, 3]]),
    y=xr.Variable(("u", "v"), [[1, 1], [2, 2]]),
)
da_points2d

## Label-based Indexing (`sel`)

Label-based indexing also preserves the RasterIndex where possible, like positional indexing.

In [ ]:
da.sel(x=slice(-2e6, 2e6), y=slice(4e6, -2e6))